In [76]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample

In [77]:

#path to target images: 
PATH_TARGET_IMAGES_FOLDER = path.Path(r'E:\eden\selected')
#path to all images: 
PATH_ALL_IMAGES_FOLDER=path.Path(r'E:\eden\all_images')
#path to save outputs
PATH_OUTPUTS=path.Path(r'C:\Users\User\Desktop\pilot\outputs')

In [78]:
#path to pilot folder : this is where the similarity_fungi_all.csv is located

similarity_df = pd.read_csv(r'E:\eden\similarity_fungi_all.csv')
similarity_df.head(5)

,Unnamed: 0,image1,image2,level_0,level_1,level_2,level_3,level_4,level_5,level_6,model,category1,category2
0,0,2237852066-222666.pkl,2237852066-222668.pkl,0.518216,0.424106,0.310927,0.313127,0.476530,0.778986,0.806504,VGG,NaN,NaN
1,1,2237852066-222666.pkl,2237852112-297894.pkl,0.535670,0.424132,0.298994,0.253748,0.283725,0.455499,0.373751,VGG,NaN,NaN
2,2,2237852066-222666.pkl,2237852154-74691.pkl,0.481043,0.395120,0.272803,0.192258,0.149006,0.293096,0.332969,VGG,NaN,NaN
3,3,2237852066-222666.pkl,2237852154-74692.pkl,0.476910,0.387269,0.250685,0.172855,0.165147,0.289664,0.338491,VGG,NaN,NaN
4,4,2237852066-222666.pkl,2237852217-297922.pkl,0.515995,0.417424,0.292037,0.240059,0.287781,0.503523,0.489363,VGG,NaN,NaN


In [79]:
similarity_df[similarity_df['image1'] == '2868486438-65656.pkl']

,Unnamed: 0,image1,image2,level_0,level_1,level_2,level_3,level_4,level_5,level_6,model,category1,category2


In [80]:
# keeping only relevent data : image names and similarity across levels 
similarity_df = similarity_df.drop(['Unnamed: 0','model','category1','category2'], axis = 1)
similarity_df.head(5)

,image1,image2,level_0,level_1,level_2,level_3,level_4,level_5,level_6
0,2237852066-222666.pkl,2237852066-222668.pkl,0.518216,0.424106,0.310927,0.313127,0.476530,0.778986,0.806504
1,2237852066-222666.pkl,2237852112-297894.pkl,0.535670,0.424132,0.298994,0.253748,0.283725,0.455499,0.373751
2,2237852066-222666.pkl,2237852154-74691.pkl,0.481043,0.395120,0.272803,0.192258,0.149006,0.293096,0.332969
3,2237852066-222666.pkl,2237852154-74692.pkl,0.476910,0.387269,0.250685,0.172855,0.165147,0.289664,0.338491
4,2237852066-222666.pkl,2237852217-297922.pkl,0.515995,0.417424,0.292037,0.240059,0.287781,0.503523,0.489363


In [81]:
# specify the levels you want to average over in each sub list. The first sublist ['level_0','level_1'] is the layers 
# we wish to average over to get the lower layers average similarity. The same goes for the second and third sublist. 
levels = [['level_0'],['level_1','level_2','level_3'],['level_4','level_5','level_6']]

In [82]:
# these parameters are for the second part of the notetbook where we create
# the actual trials from the experimental setup table
number_subjects_each_image = 10
total_number_levels = 3 
number_target_images = 6

In [83]:
# loading images from sub folders and saving the image names in a list "selected_images"
# the selected images are .jpg but the image names in similarity df are .pkl 
# so we will change all names to pkl
selected_images_paths = list(PATH_TARGET_IMAGES_FOLDER.glob("*/*.jpg"))
selected_images = [image.name.replace('.jpg','.pkl') for image in selected_images_paths]

print(f'The total number of selected images is : {len(selected_images)}')
print('The selected images names look like:')
print(selected_images[0:3])

The total number of selected images is : 60
The selected images names look like:
['2238425482-313537.pkl', '2238428357-313623.pkl', '2238481928-19949.pkl']


# PART 1 : Creating experimental setup table
#### For each selected image, we find the most similar image on average from the specified sublayers. 

In [84]:
layer_matches_0 = []
layer_matches_1 = []
layer_matches_2 = []
selected_matches = []

for target_image in selected_images:
    # list of other selected images 
    other_images = [x for x in selected_images if x != target_image]
    # keeping only relevent rows with target image 
    current_image_df = similarity_df[(similarity_df['image1'] == target_image) | (similarity_df['image2'] == target_image)]
    
    # removing rows with similarity to other target images 
    for image in other_images:
        idx_image1 = current_image_df['image1'] == image
        idx_image2 = current_image_df['image2'] == image
        total_idx = idx_image1 + idx_image2
        current_image_df = current_image_df.drop(total_idx[total_idx == True].index, axis = 0)
        
    # removing rows with similarity to matched images that already have been picked 
    for image in selected_matches:
        idx_image1 = current_image_df['image1'] == image
        idx_image2 = current_image_df['image2'] == image
        total_idx = idx_image1 + idx_image2
        current_image_df = current_image_df.drop(total_idx[total_idx == True].index, axis = 0)
        
    
    for idx,level in enumerate(levels):
        # finding max similarity row
        best_trial = current_image_df.loc[np.mean(current_image_df[level], axis = 1).idxmax()]
        # saving name of pair image (it can be in one of the two columns)
        if best_trial['image1'] != target_image:
            image_name = best_trial['image1']
        else: 
            image_name = best_trial['image2']
        
        selected_matches.append(image_name)
        # appending to relevent list 
        if idx == 0:
            layer_matches_0.append(image_name)
        elif idx == 1:
            layer_matches_1.append(image_name)
        elif idx ==2:
            layer_matches_2.append(image_name)
            
        # dropping matched image so it cant be picked again 
        current_image_df = current_image_df.drop(best_trial.name, axis = 0)

        
experiment_setup_df = pd.DataFrame(list(zip(selected_images,layer_matches_0,layer_matches_1,layer_matches_2)),columns = ['target_image','layer1','layer2','layer3'])
experiment_setup_df.head(5)

,target_image,layer1,layer2,layer3
0,2238425482-313537.pkl,2238471391-240918.pkl,2238436322-314047.pkl,2898633480-217190.pkl
1,2238428357-313623.pkl,2238499057-320266.pkl,2984615361-295509.pkl,2986408339-146952.pkl
2,2238481928-19949.pkl,2883264367-365620.pkl,2864900357-361688.pkl,2981241419-145844.pkl
3,2238572663-35822.pkl,2238584894-113161.pkl,2238571930-184143.pkl,2864903351-286874.pkl
4,2864907432-362042.pkl,2864901420-212147.pkl,2901947423-70310.pkl,2989726338-74002.pkl


In [85]:
experiment_setup_df_dup = pd.DataFrame(np.repeat(experiment_setup_df.values, number_subjects_each_image*total_number_levels, axis=0),columns = experiment_setup_df.columns)

In [86]:
experiment_setup_df_dup = experiment_setup_df_dup.replace('.pkl','.jpg',regex=True)
experiment_setup_df_dup.head(6)

,target_image,layer1,layer2,layer3
0,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg
1,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg
2,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg
3,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg
4,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg
5,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg


In [87]:
targets_dataframe=pd.DataFrame(experiment_setup_df_dup['target_image']).drop_duplicates().reset_index()
layer1_dataframe=pd.DataFrame(experiment_setup_df_dup['layer1']).drop_duplicates().reset_index()
layer1_dataframe['counter']=0
layer1_dataframe['overall_counter']=0
layer2_dataframe=pd.DataFrame(experiment_setup_df_dup['layer2']).drop_duplicates().reset_index()
layer2_dataframe['counter']=0
layer2_dataframe['overall_counter']=0
layer3_dataframe=pd.DataFrame(experiment_setup_df_dup['layer3']).drop_duplicates().reset_index()
layer3_dataframe['counter']=0
layer3_dataframe['overall_counter']=0

layer_dataframe=pd.DataFrame(experiment_setup_df_dup).drop_duplicates().reset_index()
layer_dataframe['counter']=0
layer_dataframe['overall_counter_layer1']=0
layer_dataframe['overall_counter_layer2']=0
layer_dataframe['overall_counter_layer3']=0

layer_dataframe=layer_dataframe.drop_duplicates().reset_index().drop(columns=['level_0','index'])
layer_dataframe

,target_image,layer1,layer2,layer3,counter,overall_counter_layer1,overall_counter_layer2,overall_counter_layer3
0,2238425482-313537.jpg,2238471391-240918.jpg,2238436322-314047.jpg,2898633480-217190.jpg,0,0,0,0
1,2238428357-313623.jpg,2238499057-320266.jpg,2984615361-295509.jpg,2986408339-146952.jpg,0,0,0,0
2,2238481928-19949.jpg,2883264367-365620.jpg,2864900357-361688.jpg,2981241419-145844.jpg,0,0,0,0
3,2238572663-35822.jpg,2238584894-113161.jpg,2238571930-184143.jpg,2864903351-286874.jpg,0,0,0,0
4,2864907432-362042.jpg,2864901420-212147.jpg,2901947423-70310.jpg,2989726338-74002.jpg,0,0,0,0
5,2864910426-287204.jpg,2864902304-137862.jpg,2883268412-365899.jpg,2901939433-69672.jpg,0,0,0,0
6,2874299429-139694.jpg,2986447306-296584.jpg,2981244403-72215.jpg,2883266367-365742.jpg,0,0,0,0
7,2874299429-139695.jpg,2883266388-67750.jpg,2901942443-69923.jpg,2446760917-347439.jpg,0,0,0,0
8,2874301407-363895.jpg,2446761374-124213.jpg,2986397322-371109.jpg,2901947323-70275.jpg,0,0,0,0
9,2874302418-65978.jpg,2964224310-219528.jpg,2238481198-19823.jpg,2898633321-291943.jpg,0,0,0,0


In [89]:
csv_save_location = path.Path(r'E:\eden\computed_csvs')

apearances_per_image=13
number_of_subjects=35
images_per_subject=20

iteration_counter=0
iterations_sub_ind_info=[]
do_loop=True
while do_loop:
    iteration_counter=iteration_counter+1
    all_iteration_subs_df_list=[]
    try:
        for subject in range(number_of_subjects): 
            sub_df=pd.DataFrame()
            for cur_layer in np.arange(1,4):
                for i in range(images_per_subject): 
                    subline=pd.DataFrame()
                    optional_images=layer_dataframe
                    sampling_weights= 1-(layer_dataframe[['overall_counter_layer'+str(cur_layer)]]/apearances_per_image)
                    sampling_weights[layer_dataframe['counter']==1]=0
                    sampling_weights=sampling_weights/np.sum(sampling_weights)
                    sampled_img=np.random.choice(optional_images['layer'+str(cur_layer)],size=1,replace=False,p = sampling_weights.values.flatten())
                    img_index=np.where(optional_images['layer'+str(cur_layer)].values==sampled_img)[0]
                    #print(sampled_img)
                    subline['target_image']=targets_dataframe.loc[img_index]['target_image']
                    subline['pair']=sampled_img[0]
                    subline['layer']=cur_layer
                    layer_dataframe.loc[img_index,'counter']=layer_dataframe.loc[img_index,'counter']+1
                    #print(layer1_dataframe)
                    layer_dataframe.loc[img_index,['overall_counter_layer'+str(cur_layer)]]=layer_dataframe.loc[img_index,['overall_counter_layer'+str(cur_layer)]]+1
                    #add to subject matrix: 
                    sub_df=pd.concat([sub_df,subline],axis=0)
            layer_dataframe['counter']=0
            all_iteration_subs_df_list.append(sub_df)
        do_loop=False
    except:
        print(f'iteration {iteration_counter}, failed, trying again (subject: {subject} selection ind {i}')
        iterations_sub_ind_info.append((subject,i))
        if (iteration_counter==10) or ((subject>=31) and (i>10)):
            print('focred stopped')
            do_loop=False   
        else:
        #if failed, reset the counter dataframe: 
            layer_dataframe['counter']=0
            layer_dataframe['overall_counter_layer1']=0
            layer_dataframe['overall_counter_layer2']=0
            layer_dataframe['overall_counter_layer3']=0


iteration 1, failed, trying again (subject: 0 selection ind 1
iteration 2, failed, trying again (subject: 32 selection ind 19
focred stopped


In [90]:
for sub in range(len(all_iteration_subs_df_list)-1):
    curr_sub_df=all_iteration_subs_df_list[sub]
    curr_sub_df = curr_sub_df.sample(frac=1).reset_index(drop=True)
    curr_sub_df['correct']=None

    #create the test.csv for this participants: 
    correct_column = []
    test_sub_df=curr_sub_df.copy()
    for row in test_sub_df.T:
        random_pick = np.random.randint(0,2)

        if random_pick == 0:
            correct_column.append('left')
        elif random_pick == 1:
            test_sub_df.loc[row,['target_image','pair']] = test_sub_df.loc[row,['pair','target_image']].values

            correct_column.append('right')

    test_sub_df['correct'] = correct_column
    test_sub_df.columns = ['image1', 'image2', 'layer', 'correct']
    test_sub_df = test_sub_df.sample(frac=1).reset_index(drop=True)
    test_sub_df.to_csv(csv_save_location / path.Path(f'sub_test{sub}.csv'), index=False)

    #add manipulation check in random positions (left right arrows) 
    top_index=[10,20,30,40,50]
    random_noise=[-4,-3,-2,-1,0,1,2,3,4]
    arrows=['right.jpg','left.jpg']
    for arrow in range(len(top_index)):
        chosen_noise=random.choice(random_noise)
        chosen_arrow=random.choice(arrows)
        chosen_idx=top_index[arrow]+chosen_noise - 0.5
        correct=chosen_arrow.split('.')[0]

        new_row = [chosen_arrow,None,None,correct]

        curr_sub_df.loc[chosen_idx]=new_row
        curr_sub_df=curr_sub_df.sort_index().reset_index(drop=True)

    curr_sub_df.to_csv(csv_save_location / path.Path('sub_encoding'+str(sub)+'.csv'),index=False)    

In [107]:
#selected_images_path=
selected_images=path.Path(r'E:\eden\all_images_pilot')
computed_csvs_path=path.Path(r'E:\eden\computed_csvs')

test_csvs_names=[file for file in computed_csvs_path.iterdir() if 'test' in file.name]
#check that all test images exists: 
combined_df=pd.DataFrame()
for csv in test_csvs_names:
    curr_df=pd.read_csv(csv)
    combined_df=pd.concat([combined_df,curr_df],axis=0)


    

In [119]:
all_unique_images=np.unique(combined_df[['image1','image2']].values.flatten())
print(len(all_unique_images))

#save them in a specific folder: 
all_images_location=path.Path(r'E:\eden\all_images')
all_images_save_locaiton=path.Path(r'E:\eden\all_images_pilot')


240


In [120]:
missing_images=[]
for img in all_unique_images: 
    print(img)
    curimg=Image.open(all_images_location / img)
    curimg.save(all_images_save_locaiton / img)


2237856148-989.jpg
2237857458-1324.jpg
2237857458-1325.jpg
2237858942-299356.jpg
2237944827-301472.jpg
2238117096-228518.jpg
2238328478-11678.jpg
2238425482-313537.jpg
2238428357-313623.jpg
2238434559-90406.jpg
2238436252-314037.jpg
2238436322-314047.jpg
2238471391-240918.jpg
2238479053-19457.jpg
2238480278-19661.jpg
2238480434-93829.jpg
2238480648-19714.jpg
2238480677-317399.jpg
2238480803-19737.jpg
2238480803-19739.jpg
2238480980-168129.jpg
2238481198-19821.jpg
2238481198-19822.jpg
2238481198-19823.jpg
2238481630-168218.jpg
2238481813-19930.jpg
2238481928-19949.jpg
2238499057-320266.jpg
2238516618-25604.jpg
2238521193-26285.jpg
2238531493-28260.jpg
2238544191-252422.jpg
2238550049-105691.jpg
2238550816-254000.jpg
2238555262-330312.jpg
2238560245-182099.jpg
2238563377-332054.jpg
2238571930-184143.jpg
2238572663-35822.jpg
2238573447-333826.jpg
2238573546-258454.jpg
2238576792-334659.jpg
2238577590-185467.jpg
2238578544-111698.jpg
2238580740-186218.jpg
2238584894-113161.jpg
2274829821-2

In [116]:
all_unique_images

array(['2237856148-989.jpg', '2237857458-1324.jpg', '2237857458-1325.jpg',
       '2237858942-299356.jpg', '2237944827-301472.jpg',
       '2238117096-228518.jpg', '2238328478-11678.jpg',
       '2238425482-313537.jpg', '2238428357-313623.jpg',
       '2238434559-90406.jpg', '2238436252-314037.jpg',
       '2238436322-314047.jpg', '2238471391-240918.jpg',
       '2238479053-19457.jpg', '2238480278-19661.jpg',
       '2238480434-93829.jpg', '2238480648-19714.jpg',
       '2238480677-317399.jpg', '2238480803-19737.jpg',
       '2238480803-19739.jpg', '2238480980-168129.jpg',
       '2238481198-19821.jpg', '2238481198-19822.jpg',
       '2238481198-19823.jpg', '2238481630-168218.jpg',
       '2238481813-19930.jpg', '2238481928-19949.jpg',
       '2238499057-320266.jpg', '2238516618-25604.jpg',
       '2238521193-26285.jpg', '2238531493-28260.jpg',
       '2238544191-252422.jpg', '2238550049-105691.jpg',
       '2238550816-254000.jpg', '2238555262-330312.jpg',
       '2238560245-182099.jpg